In [7]:
import pandas as pd
import numpy as np

# --- HÀM MỚI: Dọn dẹp tỉ số (phiên bản nâng cấp) ---
def clean_score(score):
    """
    Hàm nâng cấp để dọn dẹp tỉ số.
    - Xử lý giá trị rỗng (NaN).
    - Loại bỏ khoảng trắng thừa.
    - Chuẩn hóa các loại dấu gạch ngang.
    - Xử lý chữ hoa/thường ('Feb', 'feb').
    - Chuyển đổi các định dạng ngày tháng về lại tỉ số.
    """
    # 1. Bỏ qua nếu không phải là chuỗi (ví dụ: NaN, số,...)
    if not isinstance(score, str):
        return score

    # 2. Tiền xử lý: loại bỏ khoảng trắng và chuẩn hóa dấu gạch ngang
    cleaned_score = score.strip().replace('–', '-')

    # 3. Kiểm tra xem đã đúng định dạng 'số-số' chưa
    try:
        p1, p2 = cleaned_score.split('-')
        if p1.isdigit() and p2.isdigit():
            return cleaned_score # Đã đúng, trả về luôn
    except ValueError:
        # Nếu không split được, có thể là định dạng lạ, trả về giá trị gốc
        return score

    # 4. Xử lý định dạng ngày tháng
    month_map = {
        'Jan': '1', 'Feb': '2', 'Mar': '3', 'Apr': '4', 'May': '5', 'Jun': '6',
        'Jul': '7', 'Aug': '8', 'Sep': '9', 'Oct': '10', 'Nov': '11', 'Dec': '12'
    }

    # Tách lại chuỗi đã được làm sạch
    parts = cleaned_score.split('-')
    if len(parts) == 2:
        p1, p2 = parts
        # Chuyển sang dạng Title Case để xử lý cả 'Feb' và 'feb'
        p1_title, p2_title = p1.title(), p2.title()

        # Trường hợp 1: '1-Feb'
        if p1.isdigit() and p2_title in month_map:
            return f"{p1}-{month_map[p2_title]}"

        # Trường hợp 2: 'Feb-00'
        if p1_title in month_map and p2.isdigit():
            return f"{month_map[p1_title]}-{p2.lstrip('0') or '0'}"

    # 5. Nếu không khớp với bất kỳ mẫu nào, trả về giá trị gốc
    return score


# --- 1. Đọc dữ liệu ---
file_path = r"E:\project 1\data_clean\match.csv"
try:
    df = pd.read_csv(file_path, encoding='utf-8')
except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file tại đường dẫn '{file_path}'")
    exit()

# --- 2. Tạo Home/Away Team ---
df['Home Team'] = np.where(df['Venue'] == 'Home', df['Squad'], df['Opponent'])
df['Away Team'] = np.where(df['Venue'] == 'Home', df['Opponent'], df['Squad'])

# --- 3. Tách và Dọn dẹp cột Score ---
result_split = df['Result'].str.split(' ', n=1, expand=True)
df['Result_Letter'] = result_split[0]
df['Score'] = result_split[1]

# --- BƯỚC KIỂM TRA QUAN TRỌNG ---
print("--- KIỂM TRA DỮ LIỆU CỘT SCORE ---")
print("Các giá trị duy nhất TRƯỚC khi dọn dẹp:")
# Hiển thị các giá trị có vấn đề để xem trước
problematic_scores = df[df['Score'].str.contains('[a-zA-Z]', na=False)]['Score'].unique()
print(problematic_scores)

# *** ÁP DỤNG HÀM DỌN DẸP NÂNG CẤP ***
df['Score'] = df['Score'].apply(clean_score)

print("\nCác giá trị duy nhất SAU khi dọn dẹp (của các trường hợp trên):")
cleaned_scores = df[df['Score'].isin(problematic_scores) == False]['Score'].unique()
print(df[df['Score'].str.contains('[a-zA-Z]', na=False) == False]['Score'].unique())
print("-------------------------------------\n")


# --- 4. Điều chỉnh Result và Score theo Home Team ---
def flip_score(score):
    if not isinstance(score, str) or '-' not in score:
        return score
    parts = score.split('-')
    if len(parts) == 2 and parts[0].isdigit() and parts[1].isdigit():
        return f"{parts[1]}-{parts[0]}"
    return score

result_mapping = {'W': 'L', 'L': 'W', 'D': 'D'}
away_games_indices = df[df['Venue'] == 'Away'].index
df.loc[away_games_indices, 'Score'] = df.loc[away_games_indices, 'Score'].apply(flip_score)
df.loc[away_games_indices, 'Result_Letter'] = df.loc[away_games_indices, 'Result_Letter'].map(result_mapping)

# --- 5. Xóa duplicate ---
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(by='Date')
unique_match_cols = ['Season', 'Round', 'Home Team', 'Away Team']
df = df.drop_duplicates(subset=unique_match_cols, keep='first').reset_index(drop=True)
print(f"Đã xóa các dòng bị trùng lặp, số dòng còn lại: {len(df)}")

# --- 6. Tạo DataFrame cuối cùng ---
final_df = df[['Season', 'Date', 'Round', 'Home Team', 'Away Team', 'Result_Letter', 'Score']].copy()
final_df = final_df.rename(columns={'Result_Letter': 'Result'})

# --- 7. Tạo Match ID ---
start_year = final_df['Season'].str.slice(0, 4)
match_codes = final_df.index + 1
final_df['Match_ID'] = 'MATCH' + start_year + match_codes.astype(str)
cols = ['Match_ID'] + [col for col in final_df if col != 'Match_ID']
final_df = final_df[cols]

# --- 8. Hiển thị kết quả ---
print("\nDữ liệu cuối cùng sau khi xử lý (5 dòng đầu):")
print(final_df.head())
final_df['Date'] = final_df['Date'].dt.date

output_path = r"E:\project 1\data_clean\match_processed.xlsx"
final_df.to_excel(output_path, index=False)
print(f"\nĐã lưu dữ liệu đã xử lý vào file: {output_path}")

--- KIỂM TRA DỮ LIỆU CỘT SCORE ---
Các giá trị duy nhất TRƯỚC khi dọn dẹp:
[]

Các giá trị duy nhất SAU khi dọn dẹp (của các trường hợp trên):
['2-0' '0-2' '1-2' '1-0' '0-0' '0-1' '2-2' '1-3' '2-3' '3-2' '3-4' '1-1'
 '4-1' '3-1' '5-3' '6-1' '2-1' '3-0' '5-0' '4-0' '6-0' '4-2' '0-4' '0-5'
 '1-4' '3-3' '0-3' '1-5' '3-5' '2-4' '6-2' '5-1' '4-3' '0-6' '1-6' '2-6'
 '5-2' '8-0' '2-5' '9-0' '0-9' '0-8' '0-7' '2-7' '7-0' '7-2' '3-6' '6-3'
 '4-4']
-------------------------------------

Đã xóa các dòng bị trùng lặp, số dòng còn lại: 2802

Dữ liệu cuối cùng sau khi xử lý (5 dòng đầu):
     Match_ID     Season       Date        Round       Home Team  \
0  MATCH20181  2018-2019 2018-08-10  Matchweek 1  Manchester Utd   
1  MATCH20182  2018-2019 2018-08-11  Matchweek 1          Fulham   
2  MATCH20183  2018-2019 2018-08-11  Matchweek 1     Bournemouth   
3  MATCH20184  2018-2019 2018-08-11  Matchweek 1          Wolves   
4  MATCH20185  2018-2019 2018-08-11  Matchweek 1   Newcastle Utd   

        Aw

In [3]:
import pandas as pd
import numpy as np

# 1. HÀM CLEAN SCORE
def clean_score(score):
    if not isinstance(score, str):
        return score
    cleaned = score.strip().replace('–', '-')
    try:
        p1, p2 = cleaned.split('-')
        if p1.isdigit() and p2.isdigit():
            return cleaned
    except:
        return score

    month_map = {
        'JAN': '1', 'FEB': '2', 'MAR': '3', 'APR': '4', 'MAY': '5', 'JUN': '6',
        'JUL': '7', 'AUG': '8', 'SEP': '9', 'OCT': '10', 'NOV': '11', 'DEC': '12',
        'JANUARY': '1', 'FEBRUARY': '2', 'MARCH': '3', 'APRIL': '4', 'JUNE': '6',
        'JULY': '7', 'AUGUST': '8', 'SEPTEMBER': '9', 'OCTOBER': '10',
        'NOVEMBER': '11', 'DECEMBER': '12'
    }
    p1, p2 = cleaned.split('-')
    p1_up, p2_up = p1.upper(), p2.upper()
    if p1.isdigit() and p2_up in month_map:
        return f"{p1}-{month_map[p2_up]}"
    if p1_up in month_map and p2.isdigit():
        p2_num = p2.lstrip('0') or '0'
        return f"{month_map[p1_up]}-{p2_num}"
    return score

def flip_score(score):
    if not isinstance(score, str) or '-' not in score:
        return score
    p = score.split('-')
    if len(p) == 2 and p[0].isdigit() and p[1].isdigit():
        return f"{p[1]}-{p[0]}"
    return score

# 2. ĐỌC DỮ LIỆU GỐC VÀ FILE MAPPING
file_path = r"E:\project 1\data_18_25\matchlog\players_standard_matchlogs.csv"
club_path = r"E:\project 1\data_clean\club.xlsx"
season_path = r"E:\project 1\data_clean\season.xlsx"

df = pd.read_csv(file_path, encoding='utf-8')
df_club = pd.read_excel(club_path)
df_season = pd.read_excel(season_path)

# Tạo dictionary để ánh xạ
club_map = dict(zip(df_club['Squad'], df_club['Club_ID']))
season_map = dict(zip(df_season['Season'], df_season['Season_ID']))

# 3. XỬ LÝ DỮ LIỆU CƠ BẢN
df['Home Team Name'] = np.where(df['Venue'] == 'Home', df['Squad'], df['Opponent'])
df['Away Team Name'] = np.where(df['Venue'] == 'Home', df['Opponent'], df['Squad'])

# 4. --- BƯỚC ÁNH XẠ CLUB_ID VÀ LỌC DỮ LIỆU (MỚI) ---
# Ánh xạ tên đội sang ID
df['Home Team'] = df['Home Team Name'].map(club_map)
df['Away Team'] = df['Away Team Name'].map(club_map)

# Xóa các dòng nếu 1 trong 2 đội không tìm thấy ID trong file club.xlsx
initial_count = len(df)
df = df.dropna(subset=['Home Team', 'Away Team'])
print(f"Đã xóa {initial_count - len(df)} dòng do không tìm thấy Club_ID tương ứng.")

# 5. TÁCH RESULT VÀ SCORE
result_split = df['Result'].str.split(' ', n=1, expand=True)
df['Result_Letter'] = result_split[0]
df['Score'] = result_split[1].apply(clean_score)

# Lật tỉ số cho trận Away
result_map = {'W': 'L', 'L': 'W', 'D': 'D'}
away_idx = df[df['Venue'] == 'Away'].index
df.loc[away_idx, 'Score'] = df.loc[away_idx, 'Score'].apply(flip_score)
df.loc[away_idx, 'Result_Letter'] = df.loc[away_idx, 'Result_Letter'].map(result_map)

# 6. XÓA DUPLICATE
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(by='Date')
unique_cols = ['Season', 'Round', 'Home Team', 'Away Team']
df = df.drop_duplicates(subset=unique_cols, keep='first').reset_index(drop=True)

# 7. CHUẨN BỊ ĐÁNH MATCH_ID
# Lưu lại năm bắt đầu của Season trước khi ánh xạ Season_ID (để dùng cho Match_ID)
df['Start_Year'] = df['Season'].str.slice(0, 4)

# Ánh xạ Season sang Season_ID (ví dụ: 2024-2025 -> SS2024)
df['Season'] = df['Season'].map(season_map)

# Tính toán số thứ tự trận và vòng
df['Match_No'] = df.groupby(['Season', 'Round']).cumcount() + 1
df['Round_No'] = df['Round'].str.extract(r'(\d+)').astype(int)

# Tạo Match_ID
df['Match_ID'] = (
    "MATCH" + 
    df['Start_Year'] + "-" + 
    df['Round_No'].astype(str).str.zfill(2) + "-" + 
    df['Match_No'].astype(str).str.zfill(2)
)

# 8. TẠO DATAFRAME CUỐI VÀ LƯU FILE
final_df = df[['Match_ID', 'Season', 'Date', 'Round', 'Home Team', 'Away Team', 'Result_Letter', 'Score']]
final_df = final_df.rename(columns={'Result_Letter': 'Result'})
final_df['Date'] = final_df['Date'].dt.date

output_path = r"E:\project 1\data_clean\match_processed.xlsx"
final_df.to_excel(output_path, index=False)

print("Done! File đã được xử lý và lưu tại:", output_path)
print(final_df.head())

Đã xóa 64 dòng do không tìm thấy Club_ID tương ứng.
Done! File đã được xử lý và lưu tại: E:\project 1\data_clean\match_processed.xlsx
          Match_ID  Season        Date        Round Home Team Away Team  \
0  MATCH2018-01-01  SS2018  2018-08-10  Matchweek 1       MUN       LEI   
1  MATCH2018-01-02  SS2018  2018-08-11  Matchweek 1       FUL       CRY   
2  MATCH2018-01-03  SS2018  2018-08-11  Matchweek 1       BOU       CAR   
3  MATCH2018-01-04  SS2018  2018-08-11  Matchweek 1       NEW       TOT   
4  MATCH2018-01-05  SS2018  2018-08-11  Matchweek 1       WOL       EVE   

  Result Score  
0      W   2-1  
1      L   0-2  
2      W   2-0  
3      L   1-2  
4      D   2-2  


hàng tuần 

In [4]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

# ==============================================================================
# 0. ĐỊNH NGHĨA CÁC ĐƯỜNG DẪN
# ==============================================================================
BASE_DIR = r"E:\project 1"

# Đường dẫn file input/output
raw_match_logs_path = os.path.join(BASE_DIR, "data_2025_2026", "matchlog_2526", "players_standard_matchlogs.csv")
final_processed_path = os.path.join(BASE_DIR, "data_clean", "match_processed.xlsx")
new_import_path = os.path.join(BASE_DIR, "data_2025_2026", "data_import", "match_import.xlsx")
log_file_path = os.path.join(BASE_DIR, "log", "match.txt")

# --- ĐƯỜNG DẪN FILE MAPPING (MỚI) ---
club_mapping_path = os.path.join(BASE_DIR, "data_clean", "club.xlsx")
season_mapping_path = os.path.join(BASE_DIR, "data_clean", "season.xlsx")

os.makedirs(os.path.dirname(log_file_path), exist_ok=True)

# ==============================================================================
# 1. CÁC HÀM HỖ TRỢ
# ==============================================================================
def clean_score(score):
    if not isinstance(score, str): return score
    cleaned = score.strip().replace('–', '-')
    try:
        p1, p2 = cleaned.split('-')
        if p1.isdigit() and p2.isdigit(): return cleaned
    except: return score
    month_map = {
        'JAN': '1', 'FEB': '2', 'MAR': '3', 'APR': '4', 'MAY': '5', 'JUN': '6',
        'JUL': '7', 'AUG': '8', 'SEP': '9', 'OCT': '10', 'NOV': '11', 'DEC': '12',
        'JANUARY': '1', 'FEBRUARY': '2', 'MARCH': '3', 'APRIL': '4', 'JUNE': '6',
        'JULY': '7', 'AUGUST': '8', 'SEPTEMBER': '9', 'OCTOBER': '10',
        'NOVEMBER': '11', 'DECEMBER': '12'
    }
    try:
        p1, p2 = cleaned.split('-')
        p1_up, p2_up = p1.upper(), p2.upper()
        if p1.isdigit() and p2_up in month_map: return f"{p1}-{month_map[p2_up]}"
        if p1_up in month_map and p2.isdigit():
            p2_num = p2.lstrip('0') or '0'
            return f"{month_map[p1_up]}-{p2_num}"
    except: pass
    return score

def flip_score(score):
    if not isinstance(score, str) or '-' not in score: return score
    p = score.split('-')
    if len(p) == 2 and p[0].isdigit() and p[1].isdigit():
        return f"{p[1]}-{p[0]}"
    return score

# ==============================================================================
# BẮT ĐẦU XỬ LÝ
# ==============================================================================
try:
    # 2. ĐỌC FILE MAPPING
    print("Đang tải dữ liệu mapping Club và Season...")
    df_club_map = pd.read_excel(club_mapping_path)
    df_season_map = pd.read_excel(season_mapping_path)
    
    club_dict = dict(zip(df_club_map['Squad'], df_club_map['Club_ID']))
    season_dict = dict(zip(df_season_map['Season'], df_season_map['Season_ID']))

    # 3. ĐỌC VÀ XỬ LÝ DỮ LIỆU THÔ
    print(f"Bắt đầu đọc file: {raw_match_logs_path}")
    df = pd.read_csv(raw_match_logs_path, encoding='utf-8')

    # Xác định tên đội bóng tạm thời
    df['Home_Name'] = np.where(df['Venue'] == 'Home', df['Squad'], df['Opponent'])
    df['Away_Name'] = np.where(df['Venue'] == 'Home', df['Opponent'], df['Squad'])

    # --- BƯỚC ÁNH XẠ VÀ LỌC (MỚI) ---
    df['Home Team'] = df['Home_Name'].map(club_dict)
    df['Away Team'] = df['Away_Name'].map(club_dict)
    df['Season'] = df['Season'].map(season_dict)

    # Loại bỏ các dòng không ánh xạ được ID (NaN)
    initial_len = len(df)
    df = df.dropna(subset=['Home Team', 'Away Team', 'Season'])
    dropped_count = initial_len - len(df)
    if dropped_count > 0:
        print(f"Cảnh báo: Đã loại bỏ {dropped_count} dòng do không tìm thấy Club_ID hoặc Season_ID tương ứng.")

    # Xử lý Result và Score
    result_split = df['Result'].str.split(' ', n=1, expand=True)
    df['Result_Letter'] = result_split[0]
    df['Score'] = result_split[1].apply(clean_score)

    result_map = {'W': 'L', 'L': 'W', 'D': 'D'}
    away_idx = df[df['Venue'] == 'Away'].index
    df.loc[away_idx, 'Score'] = df.loc[away_idx, 'Score'].apply(flip_score)
    df.loc[away_idx, 'Result_Letter'] = df.loc[away_idx, 'Result_Letter'].map(result_map)

    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values(by='Date')
    
    unique_cols = ['Season', 'Round', 'Home Team', 'Away Team']
    df = df.drop_duplicates(subset=unique_cols, keep='first').reset_index(drop=True)

    new_data_df = df[['Season', 'Date', 'Round', 'Home Team', 'Away Team', 'Result_Letter', 'Score']]
    new_data_df = new_data_df.rename(columns={'Result_Letter': 'Result'})

    # 4. SO SÁNH VÀ CẬP NHẬT
    try:
        existing_df = pd.read_excel(final_processed_path)
        existing_df['Date'] = pd.to_datetime(existing_df['Date'])
    except FileNotFoundError:
        existing_df = pd.DataFrame()

    if not existing_df.empty:
        key_cols = ['Season', 'Round', 'Home Team', 'Away Team']
        merged = pd.merge(new_data_df, existing_df[key_cols], on=key_cols, how='left', indicator=True)
        truly_new_matches_df = merged[merged['_merge'] == 'left_only'].drop(columns=['_merge'])
    else:
        truly_new_matches_df = new_data_df

    if truly_new_matches_df.empty:
        print("Không có trận đấu mới nào để cập nhật.")
    else:
        num_new_matches = len(truly_new_matches_df)
        print(f"Tìm thấy {num_new_matches} trận đấu mới.")

        # Kết hợp và tạo lại Match_ID
        if 'Match_ID' in existing_df.columns:
            existing_df = existing_df.drop(columns=['Match_ID'])
            
        combined_df = pd.concat([existing_df, truly_new_matches_df], ignore_index=True)
        combined_df = combined_df.sort_values(by=['Date', 'Home Team']).reset_index(drop=True)

        # Logic tạo Match_ID mới (Lấy năm từ Season_ID ví dụ SS2024 -> 2024)
        combined_df['Match_No'] = combined_df.groupby(['Season', 'Round']).cumcount() + 1
        combined_df['Round_No'] = combined_df['Round'].str.extract(r'(\d+)').astype(int)
        
        # Lấy 4 chữ số năm từ Season_ID
        combined_df['Year_Part'] = combined_df['Season'].str.extract(r'(\d{4})')
        
        combined_df['Match_ID'] = (
            "MATCH" + combined_df['Year_Part'] + "-" +
            combined_df['Round_No'].astype(str).str.zfill(2) + "-" +
            combined_df['Match_No'].astype(str).str.zfill(2)
        )
        
        final_cols = ['Match_ID', 'Season', 'Date', 'Round', 'Home Team', 'Away Team', 'Result', 'Score']
        combined_df = combined_df[final_cols]
        combined_df['Date'] = combined_df['Date'].dt.date

        # Lưu file và ghi log
        new_matches_with_id_df = combined_df.tail(num_new_matches)
        
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        with open(log_file_path, 'a', encoding='utf-8') as f:
            f.write(f"[{timestamp}] Da them {num_new_matches} tran. IDs: {', '.join(new_matches_with_id_df['Match_ID'].tolist())}\n")

        combined_df.to_excel(final_processed_path, index=False)
        new_matches_with_id_df.to_excel(new_import_path, index=False)
        print(f"Hoàn tất! Đã cập nhật {final_processed_path}")

except Exception as e:
    print(f"Lỗi: {e}")

Đang tải dữ liệu mapping Club và Season...
Bắt đầu đọc file: E:\project 1\data_2025_2026\matchlog_2526\players_standard_matchlogs.csv
Tìm thấy 1 trận đấu mới.
Hoàn tất! Đã cập nhật E:\project 1\data_clean\match_processed.xlsx


In [2]:
import pandas as pd

# Đường dẫn các file
path_match = r"E:\project 1\data_clean\match_processed.xlsx"
path_season = r"E:\project 1\data_clean\season.xlsx"
path_club = r"E:\project 1\data_clean\club.xlsx"
output_path = r"E:\project 1\data_clean\match_processed1.xlsx"

# 1. Đọc dữ liệu từ các file mapping
df_season = pd.read_excel(path_season)
df_club = pd.read_excel(path_club)

# Tạo từ điển (dictionary) để ánh xạ cho nhanh
# Season -> Season_ID
season_dict = dict(zip(df_season['Season'], df_season['Season_ID']))

# Squad -> Club_ID (Lưu ý: Tên cột trong file club.xlsx là 'Squad')
club_dict = dict(zip(df_club['Squad'], df_club['Club_ID']))

# 2. Đọc file match_processed.xlsx
df_match = pd.read_excel(path_match)

# 3. Thực hiện ánh xạ dữ liệu
# Ánh xạ cột Season
df_match['Season'] = df_match['Season'].map(season_dict).fillna(df_match['Season'])

# Ánh xạ cột Home Team và Away Team
# Lưu ý: Sử dụng .strip() nếu tên đội bóng có khoảng trắng thừa
df_match['Home Team'] = df_match['Home Team'].str.strip().map(club_dict).fillna(df_match['Home Team'])
df_match['Away Team'] = df_match['Away Team'].str.strip().map(club_dict).fillna(df_match['Away Team'])

# 4. Lưu đè lại chính file đó
df_match.to_excel(path_match, index=False)

print("Đã cập nhật file match_processed.xlsx thành công!")

Đã cập nhật file match_processed.xlsx thành công!
